# Differential private analysis

In [38]:
import opendp.prelude as dp
dp.enable_features("contrib")

In [39]:
#import data from csv file
with open("C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv") as f:
    data=f.read()
print(data[0:1000])

<>:2: SyntaxWarning: invalid escape sequence '\k'
<>:2: SyntaxWarning: invalid escape sequence '\k'
C:\Users\kshub\AppData\Local\Temp\ipykernel_14328\752028492.py:2: SyntaxWarning: invalid escape sequence '\k'
  with open("C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv") as f:


ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions
1,2019-01-01,Grocery Stores/Supermarkets,8700000,ONLINE,11238.128449812339,160
1,2019-01-01,Grocery Stores/Supermarkets,0500034,ONLINE,12848.165220827457,183
2,2019-01-01,Grocery Stores/Supermarkets,110621,ONLINE,12116.165569209737,173
3,2019-01-01,Hotels/Motels,8900000,OFFLINE,7745.9988789353365,38
4,2019-01-01,Restaurants,111941,OFFLINE,6927.424754451023,173
4,2019-01-01,Restaurants,111111,OFFLINE,7513.0969230991095,187
4,2019-01-01,Restaurants,111061,OFFLINE,6337.253136719919,158
5,2019-01-01,Restaurants,8780000,ONLINE,6596.213194976057,164
5,2019-01-01,Restaurants,111321,ONLINE,7109.564110740599,177
6,2019-01-01,General Retail Stores,70675,ONLINE,8150.981488290598,81
7,2019-01-01,Drug Stores/Pharmacies,9460000,ONLINE,9029.354227509248,300
8,2019-01-01,Drug Stores/Pharmacies,70374,OFFLINE,9239.46832964432,307
9,2019-01-01,"Utilities: Electric, Gas, Water",70253,ONLINE,7241.559244978171,48
10,2019-01-0

In [40]:
privacy_unit=dp.unit_of(contributions=1)
privacy_loss=dp.loss_of(epsilon=1.0)

In [41]:
context=dp.Context.compositor(
    data=data,
    privacy_unit=privacy_unit,
    privacy_loss=privacy_loss,
    split_evenly_over=10
)

In [42]:
col_names=['ID', 'date', 'merch_category', 'merch_postal_code', 'transaction_type',
       'spendamt', 'nb_transactions']
merch_types=['Grocery Stores/Supermarkets', 'Hotels/Motels',
       'Restaurants', 'General Retail Stores', 'Drug Stores/Pharmacies',
       'Hospitals','Utilities: Electric, Gas, Water',
       'Bars/Discotheques', 'Computer Network/Information Services',
       'Airlines']

In [55]:
count_query=(
    context.query().split_dataframe(',', col_names=col_names).select_column("merch_category",str).count_by_categories(merch_types,null_category=False).laplace()
)
count_query.release()

[626775, 284458, 942172, 690929, 436603, 347781, 19, 386652, 177432, 93010]

In [61]:
Bridge_count_query=(
    context.query().split_dataframe(",",col_names=col_names).df_is_equal("merch_category", "Grocery Stores/Supermarkets").subset_by("merch_category", keep_columns=["ID"]).select_column("ID",str).count().laplace()
)
Bridge_count_query.release()

626808

# pandas analysis

In [9]:
from datetime import datetime
import numpy as np
import pandas as pd

# from utilities import *
import opendp.prelude as dp

In [10]:
dp.enable_features("contrib", "floating-point", "honest-but-curious")

# PUBLIC INFO
start_date, end_date = datetime(2020, 9, 1), datetime(2021, 3, 31)
time_col = "date"

# DATA
path = "C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv"
df = pd.read_csv(path)

<>:8: SyntaxWarning: invalid escape sequence '\k'
<>:8: SyntaxWarning: invalid escape sequence '\k'
C:\Users\kshub\AppData\Local\Temp\ipykernel_20096\3898676667.py:8: SyntaxWarning: invalid escape sequence '\k'
  path = "C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv"


In [15]:
# col_names=['ID', 'date', 'merch_category', 'merch_postal_code', 'transaction_type',
#        'spendamt', 'nb_transactions']
# merch_types=['Grocery Stores/Supermarkets', 'Hotels/Motels',
#        'Restaurants', 'General Retail Stores', 'Drug Stores/Pharmacies',
#        'Hospitals','Utilities: Electric, Gas, Water',
#        'Bars/Discotheques', 'Computer Network/Information Services',
#        'Airlines']

In [12]:
df.head()

,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions
0,1,2019-01-01,Grocery Stores/Supermarkets,8700000,ONLINE,11238.128450,160
1,1,2019-01-01,Grocery Stores/Supermarkets,500034,ONLINE,12848.165221,183
2,2,2019-01-01,Grocery Stores/Supermarkets,110621,ONLINE,12116.165569,173
3,3,2019-01-01,Hotels/Motels,8900000,OFFLINE,7745.998879,38
4,4,2019-01-01,Restaurants,111941,OFFLINE,6927.424754,173


In [13]:
df['merch_category'].value_counts()

merch_category
Restaurants                              942172
General Retail Stores                    690954
Grocery Stores/Supermarkets              626791
Drug Stores/Pharmacies                   436601
Bars/Discotheques                        386650
Hospitals                                347776
Hotels/Motels                            284449
Utilities: Electric, Gas, Water          194370
Computer Network/Information Services    177441
Airlines                                  93005
Name: count, dtype: int64

In [14]:
df["merch_category"].unique()

array(['Grocery Stores/Supermarkets', 'Hotels/Motels', 'Restaurants',
       'General Retail Stores', 'Drug Stores/Pharmacies',
       'Utilities: Electric, Gas, Water', 'Hospitals',
       'Bars/Discotheques', 'Computer Network/Information Services',
       'Airlines'], dtype=object)

In [16]:
df.columns

Index(['ID', 'date', 'merch_category', 'merch_postal_code', 'transaction_type',
       'spendamt', 'nb_transactions'],
      dtype='object')

In [17]:
df_filtered=df[((df['merch_postal_code']==7071) & (df["date"]=="2019-01-01") & (df["transaction_type"] == "OFFLINE"))]
print(len(df_filtered))
df_filtered

12


,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions
1162,573,2019-01-01,Restaurants,7071,OFFLINE,6737.048092,168
1927,947,2019-01-01,Drug Stores/Pharmacies,7071,OFFLINE,8264.313132,275
1929,948,2019-01-01,General Retail Stores,7071,OFFLINE,7480.073131,74
7234,3607,2019-01-01,General Retail Stores,7071,OFFLINE,8646.036140,86
8943,4452,2019-01-01,Drug Stores/Pharmacies,7071,OFFLINE,8826.815333,294
9354,4656,2019-01-01,General Retail Stores,7071,OFFLINE,7218.871147,72
9822,4896,2019-01-01,Hotels/Motels,7071,OFFLINE,7411.262525,37
10978,5476,2019-01-01,Bars/Discotheques,7071,OFFLINE,4959.009496,99
15032,7494,2019-01-01,Restaurants,7071,OFFLINE,7336.275632,183
17212,8592,2019-01-01,Drug Stores/Pharmacies,7071,OFFLINE,9137.248803,304


In [18]:
df_filtered=df[(df["date"]>="2019-01-01") & (df["date"]<="2019-01-08") & (df["transaction_type"] == "OFFLINE")]
print(len(df_filtered))
df_filtered

20167


,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions
3,3,2019-01-01,Hotels/Motels,8900000,OFFLINE,7745.998879,38
4,4,2019-01-01,Restaurants,111941,OFFLINE,6927.424754,173
5,4,2019-01-01,Restaurants,111111,OFFLINE,7513.096923,187
6,4,2019-01-01,Restaurants,111061,OFFLINE,6337.253137,158
11,8,2019-01-01,Drug Stores/Pharmacies,70374,OFFLINE,9239.468330,307
...,...,...,...,...,...,...,...
39990,9994,2019-01-08,Restaurants,110321,OFFLINE,6791.602852,169
39998,9999,2019-01-08,Hospitals,70640,OFFLINE,15173.174114,15
39999,9999,2019-01-08,Hospitals,111811,OFFLINE,15073.427722,15
40000,10000,2019-01-08,Restaurants,70314,OFFLINE,7281.113482,182


In [40]:
len(df["merch_postal_code"].unique())

303

In [41]:
def categorize_city(code):
    if code.startswith("5"):
        return "Medellin"
    elif code.startswith("11"):
        return "Bogota"
    elif code.startswith("70"):
        return "Brasilia"
    else:
        return "Santiago"

def location_preprocess(df):
    loc_df = df.copy()
    # Convert merchant_postal_code into str type
    loc_df["merch_postal_code"] = loc_df["merch_postal_code"].astype(str)
    # Apply the function to create a new column
    loc_df["city"] = loc_df["merch_postal_code"].apply(
        categorize_city
    )
    return loc_df

In [42]:
df_city=location_preprocess(df)

In [50]:
df_city.head()

,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions,city
0,1,2019-01-01,Grocery Stores/Supermarkets,8700000,ONLINE,11238.128450,160,Santiago
1,1,2019-01-01,Grocery Stores/Supermarkets,500034,ONLINE,12848.165221,183,Medellin
2,2,2019-01-01,Grocery Stores/Supermarkets,110621,ONLINE,12116.165569,173,Bogota
3,3,2019-01-01,Hotels/Motels,8900000,OFFLINE,7745.998879,38,Santiago
4,4,2019-01-01,Restaurants,111941,OFFLINE,6927.424754,173,Bogota


In [62]:
# filter the df_city for Bogota
df_bogota=df_city[df_city["city"]=="Bogota"]
df_bogota.head()

,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions,city
2,2,2019-01-01,Grocery Stores/Supermarkets,110621,ONLINE,12116.165569,173,Bogota
4,4,2019-01-01,Restaurants,111941,OFFLINE,6927.424754,173,Bogota
5,4,2019-01-01,Restaurants,111111,OFFLINE,7513.096923,187,Bogota
6,4,2019-01-01,Restaurants,111061,OFFLINE,6337.253137,158,Bogota
8,5,2019-01-01,Restaurants,111321,ONLINE,7109.564111,177,Bogota


In [63]:
def categorize_merchant(merch):
    # if code.startswith("5"):
    #     return "Medellin"
    # elif code.startswith("11"):
    #     return "Bogota"
    # elif code.startswith("70"):
    #     return "Brasilia"
    # else:
    #     return "Santiago"
    if merch in ['Hotels/Motels','Restaurants','Bars/Discotheques']:
        return "luxury"
    elif merch in ['Grocery Stores/Supermarkets','Drug Stores/Pharmacies','General Retail Stores','Utilities: Electric, Gas, Water','Hospitals']:
        return "essential"
    else:
        return "other"

def merchant_preprocess(df):
    loc_df = df.copy()
    # Convert merchant_postal_code into str type
    loc_df["merch_category"] = loc_df["merch_category"].astype(str)
    # Apply the function to create a new column
    loc_df["merch_super_category"] = loc_df["merch_category"].apply(
        categorize_merchant
    )
    return loc_df

In [ ]:
# col_names=['ID', 'date', 'merch_category', 'merch_postal_code', 'transaction_type',
#        'spendamt', 'nb_transactions']
# merch_types=['Grocery Stores/Supermarkets', 'Hotels/Motels',
#        'Restaurants', 'General Retail Stores', 'Drug Stores/Pharmacies',
#        'Hospitals','Utilities: Electric, Gas, Water',
#        'Bars/Discotheques', 'Computer Network/Information Services',
#        'Airlines']

In [64]:
df_bogota=merchant_preprocess(df_bogota)
df_bogota.head()


,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions,city,merch_super_category
2,2,2019-01-01,Grocery Stores/Supermarkets,110621,ONLINE,12116.165569,173,Bogota,essential
4,4,2019-01-01,Restaurants,111941,OFFLINE,6927.424754,173,Bogota,luxury
5,4,2019-01-01,Restaurants,111111,OFFLINE,7513.096923,187,Bogota,luxury
6,4,2019-01-01,Restaurants,111061,OFFLINE,6337.253137,158,Bogota,luxury
8,5,2019-01-01,Restaurants,111321,ONLINE,7109.564111,177,Bogota,luxury


In [65]:
#filter the df_bogota for luxury
df_bogota_luxury=df_bogota[df_bogota["merch_super_category"]=="luxury"]
df_bogota_luxury.head()

,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions,city,merch_super_category
4,4,2019-01-01,Restaurants,111941,OFFLINE,6927.424754,173,Bogota,luxury
5,4,2019-01-01,Restaurants,111111,OFFLINE,7513.096923,187,Bogota,luxury
6,4,2019-01-01,Restaurants,111061,OFFLINE,6337.253137,158,Bogota,luxury
8,5,2019-01-01,Restaurants,111321,ONLINE,7109.564111,177,Bogota,luxury
14,10,2019-01-01,Restaurants,110231,ONLINE,7125.455921,178,Bogota,luxury


In [55]:
#filter by merch_category airlines
df_airlines=df_bogota[df_bogota["merch_category"]=="Airlines"]
df_airlines.head()

,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions,city
39,24,2019-01-01,Airlines,111411,ONLINE,10017.690570,20,Bogota
40,24,2019-01-01,Airlines,110921,ONLINE,9547.794703,19,Bogota
548,277,2019-01-01,Airlines,110911,OFFLINE,10350.682736,20,Bogota
769,386,2019-01-01,Airlines,111511,OFFLINE,10253.574242,20,Bogota
845,424,2019-01-01,Airlines,110321,ONLINE,9396.399491,18,Bogota


In [56]:
#create groupby using date and sum the nb_transactions
df_airlines_grouped=df_airlines.groupby("date").sum()["nb_transactions"]    
df_airlines_grouped.head()

date
2019-01-01    3703
2019-01-08    3646
2019-01-15    3325
2019-01-22    3582
2019-01-29    3583
Name: nb_transactions, dtype: int64

In [54]:
#create groupby on merchant category date and city and sum nb_transactions
df_bogota_grouped=df_bogota.groupby(["merch_category","date"]).sum()["nb_transactions"].reset_index()
df_bogota_grouped.head()

,merch_category,date,nb_transactions
0,Airlines,2019-01-01,3703
1,Airlines,2019-01-08,3646
2,Airlines,2019-01-15,3325
3,Airlines,2019-01-22,3582
4,Airlines,2019-01-29,3583


In [43]:
df_city["city"].value_counts()

city
Bogota      1731080
Medellin    1067639
Brasilia     692626
Santiago     688864
Name: count, dtype: int64

In [48]:
#filter the zip codes belonging to Medellin
df_filtered=df_city[df_city["city"]=="Medellin"]
df_filtered.head()
len(df_filtered["merch_postal_code"].unique())

42

In [21]:
df_grouped = df_filtered.groupby(["merch_category", "date"]).agg({"spendamt": "sum", "nb_transactions": "sum"}).reset_index()
df_grouped[:100]

,merch_category,date,spendamt,nb_transactions
0,Airlines,2019-01-01,2.080255e+06,4060
1,Airlines,2019-01-08,1.967312e+06,3845
2,Bars/Discotheques,2019-01-01,4.767985e+06,94863
3,Bars/Discotheques,2019-01-08,4.643152e+06,92404
4,Computer Network/Information Services,2019-01-01,2.039069e+06,20218
5,Computer Network/Information Services,2019-01-08,2.430172e+06,24105
6,Drug Stores/Pharmacies,2019-01-01,9.888691e+06,329070
7,Drug Stores/Pharmacies,2019-01-08,9.848776e+06,327757
8,General Retail Stores,2019-01-01,1.325932e+07,131764
9,General Retail Stores,2019-01-08,1.416756e+07,140802


In [23]:
#find the maximum nb_transaction for each merch_category
df_cat_max=df.groupby("merch_category").agg({"nb_transactions":"max"}).reset_index()
df_cat_max

,merch_category,nb_transactions
0,Airlines,23
1,Bars/Discotheques,117
2,Computer Network/Information Services,76
3,Drug Stores/Pharmacies,454
4,General Retail Stores,94
5,Grocery Stores/Supermarkets,200
6,Hospitals,20
7,Hotels/Motels,47
8,Restaurants,205
9,"Utilities: Electric, Gas, Water",55


In [37]:
columns = ["nb_transactions"]
by = ["merch_postal_code", "date"]
s=df.groupby(by)[columns].sum()
print(s.head())
print(s.to_numpy()[0])
print(s.to_numpy().shape)
#reduce the dimension of the numpy array
s=s.to_numpy().flatten()
print(s.shape)
print(s[:10])


                              nb_transactions
merch_postal_code date                       
7071              2019-01-01             3501
                  2019-01-08             3454
                  2019-01-15             3464
                  2019-01-22             3512
                  2019-01-29             3487
[3501]
(63327, 1)
(63327,)
[3501 3454 3464 3512 3487 3520 3452 3500 3522 3475]


In [25]:
#find maximum enetry for nb_transaction category
max_entry=df_cat_max["nb_transactions"].max()
max_entry

np.int64(454)

In [138]:
# group merch_category for each merch_postal_code and date
df_grouped = df.groupby(["merch_postal_code", "date", "merch_category"]).size().reset_index(name="count")
df_grouped[0:1000]

,merch_postal_code,date,merch_category,count
0,7071,2019-02-12,Drug Stores/Pharmacies,1
1,7071,2019-02-12,Grocery Stores/Supermarkets,2
2,7071,2019-02-12,Hospitals,1
3,7071,2019-02-12,Restaurants,3
4,7071,2019-02-19,Airlines,1
...,...,...,...,...
995,7071,2021-03-30,Bars/Discotheques,1
996,7071,2021-03-30,Computer Network/Information Services,1
997,7071,2021-03-30,Drug Stores/Pharmacies,5
998,7071,2021-03-30,General Retail Stores,4


In [151]:
# df['date'] = df['date'].astype(str)
# df['merch_postal_code'] = df['merch_postal_code'].astype(str)
# df['merch_postal_code'] = df['merch_postal_code'].str.strip()
# df['date'] = df['date'].str.strip()

df_filtered=df[((df['merch_postal_code']==7071) & (df["date"]=="2019-01-01") & (df["transaction_type"] == "OFFLINE"))]
print(len(df_filtered))
df_filtered

0


In [130]:
# remove the "Online" category from the data
ddf = df[(df["merch_postal_code"] == "7071") ]
ddf[0:100]

KeyError: 'merch_postal_code'

In [ ]:
#sum spendamt and nb_transactions and group merch_category for each merch_postal_code and date
df_grouped = df.groupby(["merch_postal_code", "date"]).agg({"spendamt": "sum", "nb_transactions": "sum"}).reset_index()
df_grouped[:100]

,merch_postal_code,date,spendamt,nb_transactions
0,7071,2019-02-19,87201.561339,1596
1,7071,2019-02-26,101208.266092,1976
2,7071,2019-03-05,107336.753867,1877
3,7071,2019-03-12,68714.779829,1590
4,7071,2019-03-19,93999.910289,1902
...,...,...,...,...
95,7071,2020-12-15,72090.388933,1010
96,7071,2020-12-22,91023.927824,1669
97,7071,2020-12-29,59984.229007,865
98,7071,2021-01-05,39954.345009,308


In [ ]:
#sort df_grouped by merch_postal_code and then date
df_grouped = df_grouped.sort_values(by=["merch_postal_code", "date"])
df_grouped[:100]

,merch_postal_code,date,spendamt,nb_transactions
0,7071,2019-02-19,87201.561339,1596
1,7071,2019-02-26,101208.266092,1976
2,7071,2019-03-05,107336.753867,1877
3,7071,2019-03-12,68714.779829,1590
4,7071,2019-03-19,93999.910289,1902
...,...,...,...,...
95,7071,2020-12-15,72090.388933,1010
96,7071,2020-12-22,91023.927824,1669
97,7071,2020-12-29,59984.229007,865
98,7071,2021-01-05,39954.345009,308


In [ ]:
import pandas as pd
import opendp.prelude as dp

# Example DataFrame
data = {
    "transaction_type": ["OFFLINE", "ONLINE", "OFFLINE", "ONLINE"],
    "value": [10, 20, 30, 40]
}
df = pd.DataFrame(data)

# Define the hotspot_predictor function
def hotspot_predictor(param):
    return dp.Measurement(
        function=lambda df: df[(df["transaction_type"] == "OFFLINE")],
        privacy_map=lambda x: x  # Example privacy map, adjust as needed
    )

# Define bounds
bounds = (0.1, 10.0)

# Perform binary search
try:
    scale = dp.binary_search_param(hotspot_predictor, 1, 1.0, bounds=bounds)
    print(scale)
except dp.OpenDPException as e:
    print(f"OpenDPException: {e}")